In [1]:
import sqlite3
sql = '''
-- records definition

CREATE TABLE IF NOT EXIST records (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	sitename TEXT NOT NULL,
	county TEXT],
	aqi INTEGER,
	status TEXT,
	pm25 NUMERIC,
	date TEXT,
	lat NUMERIC,
	lon NUMERIC
);
'''
conn = sqlite3.connect('aqi.db')
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
cursor.close()
conn.close()

OperationalError: near "EXIST": syntax error

In [7]:
sql = '''
CREATE TABLE IF NOT EXISTS records (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	sitename TEXT NOT NULL,
	county TEXT,
	aqi INTEGER,
	status TEXT,
	pm25 NUMERIC,
	date TEXT,
	lat NUMERIC,
	lon NUMERIC
);
'''

import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect("AQI.db")

# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute(sql)

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

## WITH 寫法

In [16]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)
    cursor.close()



==============================================================

In [1]:
from tkinter import ttk
import tkinter as tk
from ttkthemes import ThemedTk
from tkinter.messagebox import showinfo

import sqlite3

import requests

def get_sitename()->list[str]:
    # url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
    # try:
    #     response = requests.get(url)
    #     response.raise_for_status()
    #     data = response.json()
    # except Exception as e:
    #     print('e')
    # else:
    #     sitenames = set()
    #     for items in data['records']:
    #         sitenames.add(items['sitename'])

    #     sitenames = list(sitenames)
    #     return sitenames
    conn = sqlite3.connect("AQI.db")

    with conn:
        cursor = conn.cursor()
        sql = '''
            SELECT DISTINCT sitename
            from records
        '''
        cursor.execute(sql)
        sitenames = []

        # for i in cursor.fetchall():
        #     sitenames.append(i[0])

        return [i[0] for i in cursor.fetchall()]

def get_selected_data(sitename)->list[list]:
    # url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
    # try:
    #     response = requests.get(url)
    #     response.raise_for_status()
    #     data = response.json()
    # except Exception as e:
    #     print('e')
    # else:
    #     outer_list = []
        
    #     for i in data['records']:
    #         inner_list = []
    #         if i['sitename'] == sitename:
    #             inner_list.append(i['datacreationdate'])
    #             inner_list.append(i['county'])
    #             inner_list.append(i['aqi'])
    #             inner_list.append(i['pm2.5'])
    #             inner_list.append(i['status'])
    #             inner_list.append(i['latitude'])
    #             inner_list.append(i['longitude'])
    #             outer_list.append(inner_list)

    conn = sqlite3.connect("AQI.db")

    with conn:
        cursor = conn.cursor()
        sql = '''
            SELECT date,county,aqi,pm25,status,lat,lon
            from records 
            WHERE sitename = ?
            ORDER BY date ASC
        '''
        cursor.execute( sql,(sitename ,))
        sitenames = []

        for i in cursor.fetchall():
            sitenames.append(i)

        print(sitenames)
    return sitenames
    
    
class Window(ThemedTk):
    def __init__(self,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.title('登入')
        #==============style===============
        style = ttk.Style(self)
        style.configure('TopFrame.TLabel',font=('Helvetica',20))
        #============end style===============
        
        #==============top Frame===============

        topFrame = ttk.Frame(self)
        ttk.Label(topFrame,text='空氣品質指標(AQI)(歷史資料)',style='TopFrame.TLabel').pack()
        topFrame.pack(padx=20,pady=20)
        
        #==============end topFrame===============

        #==============bottomFrame===============
        bottomFrame = ttk.Frame(self)
        sitenames = get_sitename()
        
        # sitenames.insert(0,'請選擇站點')
        
        self.selected_site = tk.StringVar()     ## 一定要加self
        sitenames_cb = ttk.Combobox(bottomFrame, textvariable= self.selected_site,state='readonly')
        sitenames_cb.configure(values=sitenames)
        sitenames_cb.set('請選擇站點')
        sitenames_cb.bind('<<ComboboxSelected>>', self.sitename_selected)   # link with event(use "bind")  diffferent from link with commend(use "commend")
        sitenames_cb.pack(side='left',anchor='n')   # anchor = 'n' 可使其靠上
        
        # define columns
        columns = ('date', 'county', 'aqi' , 'pm25' ,'status', 'lat' , 'lon' )  # 有五欄

        self.tree = ttk.Treeview(bottomFrame, columns=columns, show='headings')

        # define headings
        self.tree.heading('date', text='date')
        self.tree.heading('county', text='county')
        self.tree.heading('aqi', text='AQI')
        self.tree.heading('pm25', text='PM2.5')
        self.tree.heading('status' , text='Status')
        self.tree.heading('lat', text='Lat')
        self.tree.heading('lon', text='Lon')

        self.tree.column('date', width=120 )
        self.tree.column('county' , width=120  )
        self.tree.column('aqi', width=120 )
        self.tree.column('pm25', width=120 )
        self.tree.column('status', width=120 )
        self.tree.column('lat', width=120 )
        self.tree.column('lon', width=120 )



        self.tree.pack(side='right')


        bottomFrame.pack(expand=True,fill='x',padx=20,pady=(0,20),ipadx=10,ipady=10)
        #==============end bottomFrame===============
    
    def sitename_selected(self , event):        # link with event
        print( self.selected_site.get(),'selected')

        # 清除 Treeview 中的舊資料
        for item in self.tree.get_children():
            self.tree.delete(item)


        selected_data = get_selected_data(self.selected_site.get())
        for i in selected_data:
            self.tree.insert('', "end", values=i)


def main():
    window = Window(theme="arc")
    window.mainloop()

if __name__ == '__main__':
    main()
    

In [3]:
import requests
import sqlite3

def updated_data() -> list[list]:
    
    link = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'

    try:
        response = requests.get(link)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(e)
    
    else:
        conn = sqlite3.connect("AQI.db")
        with conn:
            cursor = conn.cursor()
            cursor.execute("DELETE FROM records")
            conn.commit()  # 確保刪除操作被提交
            
            for items in data['records']:
                sitename = items['sitename']
                county = items['county']
                aqi = int(items['aqi']) if items['aqi'] != '' else 0
                status = items['status']
                pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
                date = items['datacreationdate']
                lon = float(items['longitude']) if items['longitude'] != '' else 0.0
                lat = float(items['latitude']) if items['latitude'] != '' else 0.0
                sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                        values (?,?, ?, ?,?,?,?,?);
                '''
                cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))

            conn.commit()

updated_data()

### teacher's

In [30]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))

        
        

In [38]:
conn = sqlite3.connect("AQI.db")

with conn:
    cursor = conn.cursor()
    sql = '''
        SELECT DISTINCT sitename
        from records
    '''
    cursor.execute(sql)
    sitenames = []

    for i in cursor.fetchall():
        sitenames.append(i[0])
    
    print(sitenames)

['屏東(枋山)', '宜蘭（頭城）', '新北(樹林)', '屏東（琉球）', '臺南（麻豆）', '高雄（湖內）', '彰化（員林）', '大城', '富貴角', '麥寮', '關山', '馬公', '金門', '馬祖', '埔里', '復興', '永和', '竹山', '中壢', '三重', '冬山', '宜蘭', '陽明', '花蓮', '臺東', '恆春', '潮州', '屏東', '小港', '前鎮', '前金', '左營', '楠梓', '林園', '大寮', '鳳山', '仁武', '橋頭', '美濃', '臺南', '安南', '善化', '新營', '嘉義', '臺西', '朴子', '新港', '崙背', '斗六', '南投', '二林', '線西', '彰化', '西屯', '忠明', '大里', '沙鹿', '豐原', '三義', '苗栗', '頭份', '新竹', '竹東', '湖口', '龍潭', '平鎮', '觀音', '大園', '桃園', '大同', '松山', '古亭', '萬華', '中山', '士林', '淡水', '林口', '菜寮', '新莊', '板橋', '土城', '新店', '萬里', '汐止', '基隆']


In [6]:
conn = sqlite3.connect("AQI.db")

with conn:
    cursor = conn.cursor()
    sql = '''
        SELECT date,county,aqi,pm25,status,lat,lon
        from records 
        WHERE sitename = ?
        ORDER BY date ASC
    '''
    cursor.execute(sql,('富貴角' ,))
    sitenames = []

    for i in cursor.fetchall():
        sitenames.append(i)
    
    print(sitenames)

[('2024-11-04 08:00', '新北市', 49, 13, '良好', 25.29681695, 121.53656894), ('2024-11-04 08:00', '新北市', 49, 13, '良好', 25.29681695, 121.53656894), ('2024-11-04 09:00', '新北市', 49, 14, '良好', 25.29681695, 121.53656894), ('2024-11-04 09:00', '新北市', 49, 14, '良好', 25.29681695, 121.53656894), ('2024-11-04 10:00', '新北市', 48, 13, '良好', 25.29681695, 121.53656894), ('2024-11-04 10:00', '新北市', 48, 13, '良好', 25.29681695, 121.53656894), ('2024-11-04 11:00', '新北市', 48, '', '良好', 25.29681695, 121.53656894), ('2024-11-04 11:00', '新北市', 48, '', '良好', 25.29681695, 121.53656894), ('2024-11-04 12:00', '新北市', 48, '', '良好', 25.29681695, 121.53656894), ('2024-11-04 12:00', '新北市', 48, '', '良好', 25.29681695, 121.53656894), ('2024-11-04 13:00', '新北市', 47, 8, '良好', 25.29681695, 121.53656894), ('2024-11-04 13:00', '新北市', 47, 8, '良好', 25.29681695, 121.53656894)]
